# Rating Transitions Matrix

First thing we need to do is to:

+ reference the Thomson Reuters Eikon API; 
+ create an application ID for our script.

In [1]:
import eikon as tr
import pandas as pd

In [4]:
tr.set_app_id('your_app_id')

We are going to request the Issuer rating now and 10 years ago. We are going to get the name of the company (`TR.CommonName`), issuer rating from S&P as of now (`TR.IssuerRating(IssuerRatingSrc=SPI)`) and historical S&P rating (`TR.IssuerRating(IssuerRatingSrc=SPI,Sdate=-10Y)`)

In [5]:
ratings_data, err = tr.get_data(instruments=['0#.SPX'], 
                                fields=['TR.CommonName',
                                        'TR.IssuerRating(IssuerRatingSrc=SPI)',
                                        'TR.IssuerRating(IssuerRatingSrc=SPI,Sdate=-10Y)'])
ratings_data.head()

,Instrument,Company Common Name,Issuer Rating,Issuer Rating
0,MMM.N,3M Co,AA-,AA
1,ABT.N,Abbott Laboratories,BBB,AA
2,ABBV.N,AbbVie Inc,A-,
3,ACN.N,Accenture PLC,A+,A+
4,ATVI.OQ,Activision Blizzard Inc,BBB-,BB-


Let's build a proper readable output. For this, we need to sort ratings by their rank.

In [4]:
sp_pos_by_rating = {'AAA':1,'AA+':2,'AA':3,'AA-':4,'A+':5,'A':6,'A-':7,'BBB+':8,
            'BBB':9,'BBB-':10,'BB+':11,'BB':12,'BB-':13,'B+':14,'B':15,
            'B-':16,'CCC+':17,'CCC':18,'CCC-':19,'CC':20,'C':21,'RD':22,
            'SD':23,'D':24, 'NR':25}

Let's filter out blank values.

In [5]:
ratings = set(ratings_data.iloc[:,2].tolist() + ratings_data.iloc[:,3].tolist())
ratings_filtered = filter(None, ratings)
labels = sorted(ratings_filtered, key=lambda r: sp_pos_by_rating.get(r))

We will create a dataframe that will show us the matrix and also and a nifty dictionary `lookup`, where we can see the companies in transition.

In [6]:
result = pd.DataFrame(0, index=labels, columns=labels)
lookup = {}

for index, code, issuer, current_rating, historic_rating in ratings_data.itertuples():
    if (current_rating!='') & (historic_rating!=''):
        result[current_rating][historic_rating] += 1
        lookup.setdefault((historic_rating, current_rating), []).append(issuer)

result

,AAA,AA+,AA,AA-,A+,A,A-,BBB+,BBB,BBB-,BB+,BB,BB-,B+,B,B-,D,NR
AAA,1,1,3,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
AA+,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
AA,0,0,1,3,2,0,0,3,1,0,0,0,0,0,0,0,0,0
AA-,0,0,1,2,2,2,1,2,0,0,0,0,0,0,0,0,0,0
A+,0,0,0,6,4,10,6,6,0,0,0,0,0,0,0,0,0,0
A,0,0,0,0,7,15,15,10,3,0,0,0,0,0,0,0,0,0
A-,0,0,0,1,0,11,9,5,8,1,2,0,0,0,0,0,0,0
BBB+,0,0,0,0,0,5,6,19,15,7,0,0,0,0,0,0,0,0
BBB,0,0,0,0,0,2,11,18,11,9,4,4,0,0,0,0,0,1
BBB-,0,0,0,0,1,0,3,7,9,8,2,0,0,0,0,0,0,1


Let's test the results. Which companies ratings changed from BB- to AA-?

In [7]:
lookup[('BB-', 'AA-')]

['Amazon.com Inc']

In [8]:
lookup[('A', 'A-')]

['Aflac Inc',
 'Baxter International Inc',
 'Brown-Forman Corp',
 'Consolidated Edison Inc',
 'Dover Corp',
 'E I du Pont de Nemours and Co',
 'Ecolab Inc',
 'McCormick & Company Inc',
 'Metlife Inc',
 'NextEra Energy Inc',
 'PNC Financial Services Group Inc',
 'PPG Industries Inc',
 'Rockwell Collins Inc',
 'Southern Co',
 'United Technologies Corp']